<a href="https://colab.research.google.com/github/daegeunbae/daegeunbae/blob/main/240709_LLM_%ED%97%88%EA%B9%85%ED%8E%98%EC%9D%B4%EC%8A%A4_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D2_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자연어 처리 감정분석 전이학습

- GPU로 런타임 연결하기

In [ ]:
!pip install -qq torch transformers datasets numpy evaluate pandas

In [ ]:
!pip install -qq accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
import pandas as pd
import numpy as np
import evaluate

In [ ]:
dataset = load_dataset("sepidmnorozy/Korean_sentiment")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1333
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 2667
    })
})

In [ ]:
print(dataset['train'][3118])
print(dataset['train'][14310])

{'label': 1, 'text': '졸잼!!!성아가나중에억울한일이잇어서좀슬펏는데마지막은기쁘게끝나서다행이에여'}
{'label': 0, 'text': '진짜 어떻게 된놈의 영화가 여고괴담 1보다도 못함? 신기하다 그것도 2012년작이 1998년보다 못함 솔까 여고괴담1은 반전은 최고지 뭐 이놈의 영화는 여고괴담 시리즈보다도 못하는거같다'}


## 토큰화 Tokenize

https://huggingface.co/kykim/bert-kor-base

In [ ]:
model_name = "kykim/bert-kor-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

BertTokenizerFast(name_or_path='kykim/bert-kor-base', vocab_size=42000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
def tokenizer_func(x):
    return tokenizer(
        x['text'],
        padding="max_length",
        max_length=256,
        truncation=True
    )

In [ ]:
tokenized_datasets = dataset.map(tokenizer_func, batched=True)

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

In [ ]:
train_num_samples = 10000

train_ds = tokenized_datasets['train'].shuffle(seed=42).select(range(train_num_samples))
eval_ds = tokenized_datasets['validation'].shuffle(seed=42)

## 전이학습 Transfer Learning

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Hyperparameters

In [ ]:
bs = 32
epochs = 4
lr = 1e-5

https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

In [ ]:
args = TrainingArguments(
    'outputs',
    learning_rate=lr,
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    fp16=True,
    evaluation_strategy='epoch',
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs,
    gradient_accumulation_steps=4, # until bs=128
    eval_accumulation_steps=4,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to='none'
)

### Metrics

In [ ]:
metric = evaluate.load('accuracy')

# all Transformers models return logits
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

### Trainer

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.331078,0.870968
1,No log,0.292772,0.879220
2,No log,0.287188,0.882971
3,No log,0.290453,0.885971


TrainOutput(global_step=312, training_loss=0.342064441778721, metrics={'train_runtime': 563.1703, 'train_samples_per_second': 71.026, 'train_steps_per_second': 0.554, 'total_flos': 5247486888099840.0, 'train_loss': 0.342064441778721, 'epoch': 3.99})

In [ ]:
trainer.save_model("./mymodel")

## 추론 Inference

In [ ]:
pipe = pipeline('text-classification', model="./mymodel")

### 테스트셋 사용

In [ ]:
test_data = dataset['validation'].shuffle(seed=424)[:100]
td = pd.DataFrame(test_data)
td

,label,text
0,1,"비밀은 적정선에 관한 이야기이기도 하다. 사람에게 돈이란, 평판이란, 책임이란, 기..."
1,0,메인 시나리오 뼈대가 없고 보여주기라기엔 용과의 전투씬도 너무 부족함..더욱이 결말...
2,1,올??? 찌질한 스커드가 노만 리더스 행님이었다니 !! ㅋㅋ
3,0,5점이 적당하나 평점이 쓸데없이 높아서 낮출 필요가 있슴...
4,0,"연출, 기획, 스토리 전부다 쓰레기..제인생 영화중에 제일 쓰레기.. 절대보지마요진심"
...,...,...
95,1,"""""""철아 엄마가 꼭 니 신세 갚고 죽을께"""""""
96,0,이건 뭐 .. 완전 삼류네
97,0,으... 밑도 끝도 없음. 여자가 처음부터 자기혈청을 제공했으면 카오스가 안왔을거 아님?
98,0,요즘에 보기에는 상당히 지루한 진부한 스릴러. 원초적 본능을 뛰어 넘기에도 힘든 영...


In [ ]:
preds = pipe(td['text'].tolist())

preds_df = pd.DataFrame(preds)
preds_df

,label,score
0,LABEL_1,0.980313
1,LABEL_0,0.987016
2,LABEL_0,0.892061
3,LABEL_1,0.645307
4,LABEL_0,0.989223
...,...,...
95,LABEL_1,0.894210
96,LABEL_0,0.990540
97,LABEL_0,0.970065
98,LABEL_0,0.989042


In [ ]:
preds_df.rename(columns={'label':'pred'}, inplace=True)
preds_df['pred'] = preds_df['pred'].map({'LABEL_1': 1, 'LABEL_0': 0})

preds_df = pd.concat([preds_df, td], axis=1)
preds_df

,pred,score,label,text
0,1,0.980313,1,"비밀은 적정선에 관한 이야기이기도 하다. 사람에게 돈이란, 평판이란, 책임이란, 기..."
1,0,0.987016,0,메인 시나리오 뼈대가 없고 보여주기라기엔 용과의 전투씬도 너무 부족함..더욱이 결말...
2,0,0.892061,1,올??? 찌질한 스커드가 노만 리더스 행님이었다니 !! ㅋㅋ
3,1,0.645307,0,5점이 적당하나 평점이 쓸데없이 높아서 낮출 필요가 있슴...
4,0,0.989223,0,"연출, 기획, 스토리 전부다 쓰레기..제인생 영화중에 제일 쓰레기.. 절대보지마요진심"
...,...,...,...,...
95,1,0.894210,1,"""""""철아 엄마가 꼭 니 신세 갚고 죽을께"""""""
96,0,0.990540,0,이건 뭐 .. 완전 삼류네
97,0,0.970065,0,으... 밑도 끝도 없음. 여자가 처음부터 자기혈청을 제공했으면 카오스가 안왔을거 아님?
98,0,0.989042,0,요즘에 보기에는 상당히 지루한 진부한 스릴러. 원초적 본능을 뛰어 넘기에도 힘든 영...


In [ ]:
mask = preds_df['pred'] == preds_df['label']

len(preds_df[mask])

91

### 내 데이터셋

In [ ]:
txts = [
    {'label': 0, 'text': "절대로 강추할 수 없는 영화"},
    {'label': 0, 'text': "절대로 추천할 수 없는 영화"},
    {'label': 1, 'text': "또 보고 싶다."},
    {'label': 0, 'text': "이걸 보면서 웃을 수는 없다."},
    {'label': 0, 'text': "처음에는 재미있었는데 갈수록 산으로 가는 내용."},
    {'label': 1, 'text': "요즘 재미없는 영화만 나오는데 신선한 충격을 준 영화."},
    {'label': 1, 'text': "유명한 감독이나 배우가 나오지는 않지만 스토리가 감동"}
]

txts_td = pd.DataFrame(txts)
txts_td

,label,text
0,0,절대로 강추할 수 없는 영화
1,0,절대로 추천할 수 없는 영화
2,1,또 보고 싶다.
3,0,이걸 보면서 웃을 수는 없다.
4,0,처음에는 재미있었는데 갈수록 산으로 가는 내용.
5,1,요즘 재미없는 영화만 나오는데 신선한 충격을 준 영화.
6,1,유명한 감독이나 배우가 나오지는 않지만 스토리가 감동


In [ ]:
preds_txts = pipe(txts_td['text'].tolist())

In [ ]:
preds_txts_df = pd.DataFrame(preds_txts)
preds_txts_df.rename(columns={'label':'pred'}, inplace=True)
preds_txts_df['pred'] = preds_txts_df['pred'].map({'LABEL_1': 1, 'LABEL_0': 0})

preds_txts_df = pd.concat([preds_txts_df, txts_td], axis=1)
preds_txts_df

,pred,score,label,text
0,1,0.925147,0,절대로 강추할 수 없는 영화
1,0,0.961567,0,절대로 추천할 수 없는 영화
2,1,0.980364,1,또 보고 싶다.
3,0,0.939811,0,이걸 보면서 웃을 수는 없다.
4,0,0.930401,0,처음에는 재미있었는데 갈수록 산으로 가는 내용.
5,1,0.961845,1,요즘 재미없는 영화만 나오는데 신선한 충격을 준 영화.
6,1,0.957517,1,유명한 감독이나 배우가 나오지는 않지만 스토리가 감동


### 파이프라인 사용하지 않고 모델 로딩

In [ ]:
model_inf = AutoModelForSequenceClassification.from_pretrained("./mymodel")
inputs = tokenizer(txts_td['text'].tolist(), padding=True, return_tensors="pt")

with torch.no_grad():
    logits = model_inf(**inputs).logits

In [ ]:
logits.argmax(axis=1)

tensor([1, 0, 1, 0, 0, 1, 1])